In [3]:
import pandas as pd
import geopandas as gpd
import os
from shapely import wkt


In [13]:
df = pd.concat([pd.read_csv('../brasil/data/20230124/sao_paolo_A.csv'),
                pd.read_csv('../brasil/data/20230124/sao_paolo_B.csv'),
                pd.read_csv('../brasil/data/20230124/sao_paolo_C.csv'),
                pd.read_csv('../brasil/data/20230124/sao_paolo_D.csv')])
df.drop_duplicates(subset='id').to_csv('../brasil/data/20230124/sao_paolo.csv',index=False)

In [36]:
def concat_csv_files(csv_path, geojson_path,output_path_history):
    
    file_dict = {}
    for root, dirs, files in os.walk(csv_path):
        for file in files:
            if file.endswith(".csv"):
                file_name = file.split(".csv")[0]
                geojson_file = f'{file_name.lower()}_geometries.csv'
                geojson_file_path = os.path.join(geojson_path, geojson_file)
                if os.path.exists(geojson_file_path):
                    print(file_name)
                    file_path = os.path.join(root, file)
                    if file_name not in file_dict:
                        file_dict[file_name] = []
                    file_dict[file_name].append(pd.read_csv(file_path))


    # Concatenate all the dataframes in each list
    print(file_dict.keys())
    for file_name in file_dict:
            df = pd.concat(file_dict[file_name], axis=0, ignore_index=True)
            geojson_file = f'{file_name.lower()}_geometries.csv'
            geojson_file_path = os.path.join(geojson_path, geojson_file)
            

            gdf_geom = pd.read_csv(geojson_file_path)
            gdf_geom['geometry'] = gdf_geom['geometry'].apply(wkt.loads)
            gdf_geom = gpd.GeoDataFrame(gdf_geom, crs='epsg:4326')

            gdf = gpd.GeoDataFrame(df[['lat','lon','$m2','scrapingTime']],geometry=gpd.points_from_xy(df.lon,df.lat),crs='epsg:4326')
            gdf.drop(columns=['lat','lon'],inplace=True)
            df_history = gdf.sjoin(gdf_geom).groupby(['scrapingTime','name']).agg(ads=('$m2','count'),avgPrice=('$m2','mean')).reset_index()
            df_history.sort_values(['name','scrapingTime'],inplace=True)
            os.makedirs(output_path_history,exist_ok=True)
            df_history.to_csv(os.path.join(output_path_history,file_name+'_historicals.csv'),index=False)

# Test the function with sample paths
country = 'colombia'
csv_path = f'../{country}/data'
geojson_path = f'../{country}/geojson'
output_path_history = f'../{country}/data/historicals'
concat_csv_files(csv_path, geojson_path,output_path_history)


Bogota


/tmp/ipykernel_997/875795463.py:15: DtypeWarning: Columns (33,50,72,73,74,75,76,77,78,79,80,81,82,91,92,97,101,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,124,125,126,127,128,129,130,132,133,134,136,137,138) have mixed types. Specify dtype option on import or set low_memory=False.
  file_dict[file_name].append(pd.read_csv(file_path))


Bogota


/tmp/ipykernel_997/875795463.py:15: DtypeWarning: Columns (43,50,60,73,74,75,76,77,78,79,80,81,82,83,96,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,123,124,125,127,128,129,130,131,132,133,135,136,137,138,139,140,141,142,143,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  file_dict[file_name].append(pd.read_csv(file_path))


Bogota


/tmp/ipykernel_997/875795463.py:15: DtypeWarning: Columns (43,50,60,71,76,77,78,79,80,81,82,83,96,100,101,102,103,104,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,123,124,125,127,128,129,130,131,132,133,134,135,136,137,139,140,141,142,143,144,145,146,147,148,149,150) have mixed types. Specify dtype option on import or set low_memory=False.
  file_dict[file_name].append(pd.read_csv(file_path))


Bogota


/tmp/ipykernel_997/875795463.py:15: DtypeWarning: Columns (43,50,64,71,74,75,76,77,78,79,80,81,82,83,96,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,124,125,126,127,128,129,130,132,133,134,136,137,138) have mixed types. Specify dtype option on import or set low_memory=False.
  file_dict[file_name].append(pd.read_csv(file_path))


dict_keys(['Bogota'])


In [5]:
df

,name,geometry
0,Abranches,"POLYGON ((-49.2808111 -25.3537107, -49.2807949..."
1,Afonso Pena,"POLYGON ((-49.2017183 -25.5136346, -49.2013174..."
2,Ahú,"POLYGON ((-49.2716195 -25.4073394, -49.2714916..."
3,Alto Boqueirão,"POLYGON ((-49.2579071 -25.528812, -49.2575849 ..."
4,Alto da Glória,"POLYGON ((-49.2678268 -25.4215837, -49.2676172..."
...,...,...
83,Vila Izabel,"POLYGON ((-49.3037079 -25.4550368, -49.2943751..."
84,Vista Alegre,"POLYGON ((-49.3070715 -25.4147435, -49.3070268..."
85,Weissópolis,"POLYGON ((-49.2005482 -25.4431651, -49.2001118..."
86,Xaxim,"POLYGON ((-49.2848309 -25.5073907, -49.2836897..."


/home/gianluca/anaconda3/envs/cp/lib/python3.8/site-packages/geopandas/geodataframe.py:2061: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


,scrapingTime,name,ads,avgPrice
0,2023-01-25,Abranches,86,978.645107
86,2023-01-30,Abranches,80,1076.727251
173,2023-02-06,Abranches,88,967.778097
260,2023-02-13,Abranches,91,985.343996
1,2023-01-25,Afonso Pena,461,860.918043
...,...,...,...,...
345,2023-02-13,Xaxim,929,844.513070
85,2023-01-25,Água Verde,727,1716.058169
172,2023-01-30,Água Verde,684,1849.041716
259,2023-02-06,Água Verde,723,1721.848519


In [21]:
df_history

,scrapingTime,name,ads,avgPrice
0,2023-01-25,Abranches,86,978.645107
1,2023-01-25,Afonso Pena,461,860.918043
2,2023-01-25,Ahú,472,1776.138346
3,2023-01-25,Alto Boqueirão,392,861.514063
4,2023-01-25,Alto da Glória,290,2125.864293
...,...,...,...,...
342,2023-02-13,Vila Izabel,244,1688.603203
343,2023-02-13,Vista Alegre,136,970.513157
344,2023-02-13,Weissópolis,69,693.383011
345,2023-02-13,Xaxim,929,844.513070
